# Course Code: DS4003 
# Course Name: Principles and Techniques for Data Science 
# Homework Assignment 3
# Due date: 6th December 2024
## Save the notebook with your name and submit it by the due date.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

## Data overview

This dataset contains information on the quality of the air as measured by air quallity instruments for a period in time.


In [ ]:
air_data= pd.read_csv('air_quality.csv')    ## Dont forget to remove negative values

air_data

,Date,Time,NOx(GT),PT08.S1(CO)
0,2004-03-10,18:00:00,166.0,1360.00
1,2004-03-10,19:00:00,103.0,1292.25
2,2004-03-10,20:00:00,131.0,1402.00
3,2004-03-10,21:00:00,172.0,1375.50
4,2004-03-10,22:00:00,131.0,1272.25
...,...,...,...,...
9352,2005-04-04,10:00:00,471.7,1314.25
9353,2005-04-04,11:00:00,353.3,1162.50
9354,2005-04-04,12:00:00,293.0,1142.00
9355,2005-04-04,13:00:00,234.5,1002.50


<div class='alert alert-block alert-success' style="font-weight:bolder">

### Task 1 (0.1pts)
    
### 1a: Determine whether the data needs any preprocessing. If so implement the appropriate functions to preprocess it.

</div>

In [8]:
## YOUR CODE HERE

<div class='alert alert-block alert-success' style="font-weight:bolder">

### Task 2 (0.2pts)
### 2a: Break down the time series data to reveal its underlying components, such as general trends, recurring seasonal patterns, and any irregular fluctuations.
    
### 2b: Determine whether there are signs of significant shifts over time. Use an appropriate test to confirm the stationarity of the data and explain its implications for forecasting
</div>

In [ ]:
## YOUR CODE HERE

<div class='alert alert-block alert-success' style="font-weight:bolder">

### Task 3 (0.3pts)
### 3a: Forecast the values of the timeseries signals for the next 3 months from the last date. Use two different forecasting methods of your choice. Evaluate the quality of your predictions and visualise the predictions from the different methods.  Which method seems more realistic based on the observed trends and patterns in the data?

</div>

In [ ]:
## YOUR CODE HERE

### Plot the original data under the forecast

<div class='alert alert-block alert-success' style="font-weight:bolder">

### Task 4 (0.4pts)
### To understand the relationship between the variables in your dataset, your task is to build a model that captures this relationship using linear regression.
### 4a: Fit a linear regression model to your data. Derive the Coefficients Using OLS or Gradient Descent.
### 4b: Plot the regression line alongside the original data points to visually assess the fit of the model..

</div>

In [6]:
## YOUR CODE HERE

<div class='alert alert-block alert-success' style="font-weight:bolder">

### Optional Task
### Transfrom the signals using PAA or SAX

</div>

In [ ]:
## Extra credit